Description : Anomaly Detection is a binary classification identifying unusual or unexpected patterns in a dataset, which deviate significantly from the majority of the data. The goal of anomaly detection is to identify such anomalies, which could represent errors, fraud, or other types of unusual events, and flag them for further investigation, in the provided dataset, there are 12 industrial products, all labeled with normal or abnormal, the products which are 

The VisA dataset contains 12 subsets corresponding to 12 different objects as shown in the above figure. There are 10,821 images with 9,621 normal and 1,200 anomalous samples. Four subsets are different types of printed circuit boards (PCB) with relatively complex structures containing transistors, capacitors, chips, etc. For the case of multiple instances in a view, we collect four subsets: Capsules, Candles, Macaroni1 and Macaroni2. Instances in Capsules and Macaroni2 largely differ in locations and poses. Moreover, we collect four subsets including Cashew, Chewing gum, Fryum and Pipe fryum, where objects are roughly aligned. The anomalous images contain various flaws, including surface defects such as scratches, dents, color spots or crack, and structural defects like misplacement or missing parts.

# Objective
The objective of this notebook is to build a deep learning model to detect anomalies in the VisA dataset. The model will be trained on the normal images and then tested on the normal and abnormal images. The model will be evaluated based on the F1 score.

# Approach
The approach is as follows:
1. Load the data
2. Preprocess the data
3. Build the model
4. Train the model
5. Evaluate the model

# References
1. [VisA: A Dataset for Anomaly Detection in Industrial Visual Inspection](https://arxiv.org/abs/2103.04517)
2. [Anomaly Detection in Industrial Visual Inspection: A Survey](https://arxiv.org/abs/2103.04517)
3. [Anomaly Detection in Industrial Visual Inspection: A Survey](https://arxiv.org/abs/2103.04517)
4. [Anomaly Detection in Industrial Visual Inspection: A Survey](https://arxiv.org/abs/2103.04517)
5. [Anomaly Detection in Industrial Visual Inspection: A Survey](https://arxiv.org/abs/2103.04517)

# Contents
1. [Load the data](#1.-Load-the-data)
2. [Preprocess the data](#2.-Preprocess-the-data)
3. [Build the model](#3.-Build-the-model)
4. [Train the model](#4.-Train-the-model)
5. [Evaluate the model](#5.-Evaluate-the-model)
6. [Make predictions](#6.-Make-predictions)
7. [Conclusion](#7.-Conclusion)
8. [References](#8.-References)


# 1. Load the data



In [4]:



# Download the data
    # The data is available at: https://amazon-visual-anomaly.s3.us-west-2.amazonaws.com/VisA_20220922.tar
    # The data is in the form of a tar file. We will download the tar file and extract the contents.
    # The data tree of the downloaded data is as follows.
"""
        VisA
        |-- candle
        |-----|--- Data
        |-----|-----|----- Images
        |-----|-----|--------|------ Anomaly 
        |-----|-----|--------|------ Normal 
        |-----|-----|----- Masks
        |-----|-----|--------|------ Anomaly 
        |-----|--- image_anno.csv
        |-- capsules
        |-----|----- ...
    """
    # image_annot.csv gives image-level label and pixel-level annotation mask for each image. 
    #The id2class map functions for multi-class masks can be found in ./utils/id2class.py Here the masks for normal images are not stored to save space.


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np




ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

# Download the data
!wget https://amazon-visual-anomaly.s3.us-west-2.amazonaws.com/VisA_20220922.tar

# Define the data directory
!mkdir VisA
data_dir = 'VisA'

# Extract the data into a file named VisA
!tar -xvf VisA_20220922.tar -C VisA


In [3]:

# Define the image directory
image_dir = os.path.join(data_dir, 'candle/Data/Images')

# Define the image annotation file
image_anno_file = os.path.join(data_dir, 'candle/image_anno.csv')

NameError: name 'data_dir' is not defined

In [ ]:
import pandas as pd

# Load the image annotation file
image_anno = pd.read_csv(image_anno_file)

# Display the first few rows of the image annotation file
image_anno

# 2. Preprocess the data

In [ ]:
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

def resize_image(image, size=(224, 224)):
    # This function resizes the image to the given size.
    image = cv2.resize(image, size)
    return image

def normalize_image(image):
    
    #This function normalizes the image.
    
    image = image / 255.0
    return image

In [1]:
# for each image in the image directory we will resize the image to the size (256, 256) and normalize the image.
# We will store the resized and normalized image in the image list respectively.

images = []

for image in image_anno['image']:
    image_path = os.path.join(image_dir, image)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image)
    image = normalize_image(image)
    images.append(image)

# Convert the images list to a numpy array
images = np.array(images)

# Display the shape of the images array
images.shape

NameError: name 'image_anno' is not defined

In [ ]:
# Label Handling
# The image_anno file contains a lable column that indicates whether the image is normal or anomalous.
# We will convert the labels into 0 and 1 where 0 indicates normal and 1 indicates anomalous.
# the normal images are labeled "noemal" and the anomalous images are labeled with any other str.

# Define the labels
labels = np.where(image_anno['label'] == 'normal', 0, 1)

# Display the labels
labels

# replace the label column in the image_anno dataframe with the new labels
image_anno['label'] = labels







# Feature extraction

In [ ]:


# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
images_train, images_val = train_test_split(images, test_size=0.2, random_state=42)

# split the labels into training and validation sets
labels_train, labels_val = train_test_split(labels, test_size=0.2, random_state=42)

# Display the shape of the training and validation sets
print("Training set shape:", images_train.shape)
print("Validation set shape:", images_val.shape)

# Data Augmentation (Optional): We can use data augmentation to increase the diversity of the training data.
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use the data generator to create batches of augmented images during training
# Define the batch size
batch_size = 32

# Create a data generator for the training set
train_datagen = datagen.flow(images_train, batch_size=batch_size)

# Create a data generator for the validation set
val_datagen = datagen.flow(images_val, batch_size=batch_size)




In [ ]:
# load a pre-trained model
# model : vgg16

from tensorflow.keras.applications import VGG16  # Import the VGG16 model

# Load the pre-trained model without the top layers (classification head)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (optional, to prevent retraining pre-trained weights)
base_model.trainable = False

# Use the base model for feature extraction
def extract_features(img):
    img = np.expand_dims(img, axis=0)  # Add a batch dimension
    features = base_model.predict(img)  # Pass the image through the model
    return features.flatten()  # Flatten the feature vector

# Extract features from the images
features_train = np.array([extract_features(img) for img in images_train])
features_train = np.asarray(features_train).astype('float32')
features_val = np.array([extract_features(img) for img in images_val])
features_val = np.asarray(features_val).astype('float32')

# Display the shape of the extracted features
features_train.shape, features_val.shape





In [ ]:
# Define the model

from tensorflow.keras.models import Sequential


model = Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(features_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()






In [ ]:
print("feature train :")
print(features_train)
print("feature val :")
print(features_val) 



In [1]:
# Train the model
history = model.fit(features_train, labels_train, validation_data=(features_val, labels_val), epochs=10, batch_size=32)


# Evaluate the model
loss, accuracy = model.evaluate(features_val, np.ones(features_val.shape[0]) )
print("Loss:", loss)
print("Accuracy:", accuracy)

NameError: name 'model' is not defined

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Plot the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Predict the labels for the validation set
preds = model.predict(features_val)

# Convert the predictions to binary labels
pred_labels = np.where(preds > 0.5, 1, 0)

# Display the predicted labels
pred_labels

# Display the true labels
labels_val

# Calculate the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(labels_val, pred_labels)

# Display the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
# test the model on a new image
# Load a new image
new_image_path = 'VisA/candle/Data/Images/Anomaly/Anomaly_0001.jpg'
new_image = cv2.imread(new_image_path)
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = resize_image(new_image)
new_image = normalize_image(new_image)

# Extract features from the new image
new_features = extract_features(new_image)

# Reshape the features
new_features = new_features.reshape(1, -1)

# Predict the label for the new image
new_pred = model.predict(new_features)

# Convert the prediction to a binary label
new_pred_label = np.where(new_pred > 0.5, 1, 0)

# Display the new image and the predicted label
plt.imshow(new_image)
plt.axis('off')
plt.title('Predicted Label: {}'.format(new_pred_label[0]))
plt.show()



In [ ]:
# Save the model
model.save('anomaly_detection_model.h5')

# Save the feature extraction model
base_model.save('feature_extraction_model.h5')

